## Raw, Async Requests

For this walk through, we'll use the low-level bits of `BrittleWit` to get the last three things Jack tweeted.

In [1]:
import brittle_wit as bw
from brittle_wit import rest_api


bw.monkey_patch_ipython_disp()


APP_CRED = bw.AppCredentials.load_from_env()
CLIENT_CRED = bw.ClientCredentials.load_from_env()

This request is a `TwitterRequest`, like in the last section.

In [2]:
twitter_req = rest_api.statuses.user_timeline(screen_name='Jack', count=3)
twitter_req

We'll need a few things from other packages and a few from `brittle_wit` that you're unlikely to use frequently.

In [3]:
import asyncio
import aiohttp
from brittle_wit.rate_limit import RateLimit
from brittle_wit.executors import (twitter_req_to_http_req,
                                   execute_req)

Since we haven't made prior requests in this script, we don't know how many api requests remain. (I could have used them all up, then restarted the notebook!) Therefore, we'll initialize it from ignorance, which gives us one request.

In [4]:
rate_limit = RateLimit.from_ignorance()
rate_limit

RateLimit(limit=1, remaining=1, reset_time=14:50:19)

Now, let's create a coroutine that makes the request. 

https://aiohttp.readthedocs.io/en/stable/client_reference.html

In [5]:
async def jacks_last_three_tweets(client, rate_limit):
    http_req = twitter_req_to_http_req(client, 
                                       APP_CRED, 
                                       CLIENT_CRED, 
                                       twitter_req)

    return await execute_req(CLIENT_CRED, 
                             twitter_req, 
                             http_req, 
                             rate_limit)
    
async def simple_executor(loop, rate_limit):
    opts = dict(loop=loop, 
                cookie_jar=aiohttp.helpers.DummyCookieJar(),
                conn_timeout=60,
                read_timeout=60)
    async with aiohttp.ClientSession(**opts) as client:
        return await jacks_last_three_tweets(client, rate_limit)

In [6]:
loop = asyncio.get_event_loop()
coro = simple_executor(loop, rate_limit)
resp = loop.run_until_complete(coro)
texts = [status['text'] for status in resp.body]
texts

['RT @bohan: #DefendDACA #withDREAMers https://t.co/OSmLBdW5QA',
 'RT @NewsHour: Trump’s decision to end DACA, explained (via @DanielBush) \nhttps://t.co/lmwzu4hKlY',
 'Completely unnecessary and cruel. Ending #DACA is a crushing blow to those who want to contribute to our future. https://t.co/Cpu0J01ioK']